# Functional implementation:

In [1]:
import numpy as np
import pandas as pd
import nltk
import re
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

nltk.download('punkt')
nltk.download('stopwords')

import warnings  #Hide Warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
#Show text full width
pd.set_option('display.max_colwidth', None)
%pprint

Pretty printing has been turned OFF


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AliiPmD\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AliiPmD\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv('test.csv')
print(train.shape)
print(test.shape)

(7613, 5)
(3263, 4)


In [3]:
def featureAdding(df):
    df['len'] = df.text.apply(lambda x : len(x))
    df['words'] = df.text.apply(lambda x: len(nltk.word_tokenize(x)))
    df['numbers'] = df.text.apply(lambda x : len(re.findall( r'\d+,*\d+|\d+' , x)))
    df['sent'] = df.text.apply(lambda x: len(nltk.sent_tokenize(x)))
    df['hashtags'] = df.text.apply(lambda x : len(re.findall( r'#' , x)))
    # extract hashtags
    df['tags']=df.text.str.lower().apply(lambda x: " ".join([i for i in re.findall(r'#([A-Za-z]+)',x)]))
    df['mentions'] = df.text.apply(lambda x : len(re.findall( r'@' , x)))
    # extract mentions
    df['mentionText']=df.text.str.lower().apply(lambda x: " ".join([i for i in re.findall(r'@([A-Za-z0-9_.]+)',x)]))
    df['links'] = df.text.apply(lambda x : len(re.findall( r'http' , x)))
    df['density'] = df.words / df.len
    # Punctuation count per tweet lenght
    df['punctuation'] = df.text.apply(lambda x : len(re.findall( r'[^A-Za-z0-9\s]' , x))/len(x) )
    # Uppercase letters count per tweet lenght
    df['upper'] = df.text.apply(lambda x : len(re.findall( r'[A-Z]' , x))/len(x))
    # Scale
    scaler = MinMaxScaler()
    df[['len','words','numbers','sent','hashtags','mentions','links']] = scaler.fit_transform(
        df[['len','words','numbers','sent','hashtags','mentions','links']])
    return df

In [4]:
def processTweet(df):
    #lower Case:
    df['tweet'] = df.text.apply(lambda x: x.lower())
    #Remove urls:
    df['tweet'] = df['tweet'].str.replace(r'http[^\s]+'," ")
    # Remove Punctuation
    df['tweet'] = df['tweet'].str.replace(r'[^A-Za-z0-9\s]',"")
    #Remove short words <=2 chars
    df['tweet'] = df['tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
    #remove stopwords
    stopwords = nltk.corpus.stopwords.words('english')
    df['tweet'] = df['tweet'].apply(lambda x: " ".join(
        [i for i in nltk.word_tokenize(x) if i not in stopwords]))
    # Lemmatization
    lma = nltk.WordNetLemmatizer()
    df['tweet']=df['tweet'].apply(lambda x: " ".join(
        [lma.lemmatize(i) for i in nltk.word_tokenize(x)]))
    #get dummy for keyword column
    df = pd.concat([df.drop('keyword',axis=1) ,
                         pd.get_dummies(df['keyword'],dummy_na=True)], axis=1)
    
    return df

In [5]:
def removeRepetitive(df):
    tfidf = TfidfVectorizer(min_df=1).fit_transform(df['tweet'])
    pairwise_similarity= tfidf * tfidf.T
    pairwise_similarity.setdiag(np.nan)
    similarTweets = df.iloc[list(set(pairwise_similarity.indices[pairwise_similarity.data >= 0.95]))]
    df.drop(similarTweets.index , axis=0 , inplace=True)
    return df

In [6]:
def tfidf(df,isTrain):
    if isTrain:
        global vectTweet 
        vectTweet = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(df['tweet'])
        global vectTags 
        vectTags = TfidfVectorizer(min_df=2).fit(df['tags'])
        global vectMentions 
        vectMentions = TfidfVectorizer(min_df=2).fit(df['mentionText'])
  
    dfVect = vectTweet.transform(df['tweet'])
    df_tags = vectTags.transform(df['tags'])
    df_mentions = vectMentions.transform(df['mentionText'])
    df = df.reset_index()
    df.drop('index',inplace=True,axis=1)
    df_all = pd.concat([df.drop(['tags','tweet','mentionText'],axis=1),
                         pd.DataFrame.sparse.from_spmatrix(df_tags),
                         pd.DataFrame.sparse.from_spmatrix(dfVect),
                         pd.DataFrame.sparse.from_spmatrix(df_mentions)],axis=1)
    return df_all

In [7]:
def pipeline (df, isTrain:bool):
    if isTrain:
        duplicatdDF = train[train.text.str.lower().duplicated()].sort_values(by='text')
        train.drop(duplicatdDF.index , axis=0, inplace=True)
    newDF = featureAdding(df)
    newDF = processTweet(newDF)
    if isTrain:
        newDF = removeRepetitive(newDF)
    newDF = tfidf(newDF,isTrain)
    newDF.drop(['id','location','text'],axis=1,inplace=True)
    return newDF

In [8]:
newTrain = pipeline(train,True)
print(newTrain.shape)
newTrain.head(1)

(6540, 3444)


,target,len,words,numbers,sent,hashtags,mentions,links,density,punctuation,upper,ablaze,accident,aftershock,airplane%20accident,ambulance,annihilated,annihilation,apocalypse,armageddon,army,arson,arsonist,attack,attacked,avalanche,battle,bioterror,bioterrorism,blaze,blazing,bleeding,blew%20up,blight,blizzard,blood,bloody,blown%20up,body%20bag,body%20bagging,body%20bags,bomb,bombed,bombing,bridge%20collapse,buildings%20burning,buildings%20on%20fire,burned,burning,burning%20buildings,bush%20fires,casualties,casualty,catastrophe,catastrophic,chemical%20emergency,cliff%20fall,collapse,collapsed,collide,collided,collision,crash,crashed,crush,crushed,curfew,cyclone,damage,danger,dead,death,deaths,debris,deluge,deluged,demolish,demolished,demolition,derail,derailed,derailment,desolate,desolation,destroy,destroyed,destruction,detonate,detonation,devastated,devastation,disaster,displaced,drought,drown,drowned,drowning,dust%20storm,earthquake,electrocute,electrocuted,emergency,emergency%20plan,emergency%20services,engulfed,epicentre,evacuate,evacuated,evacuation,explode,exploded,explosion,eyewitness,famine,fatal,fatalities,fatality,fear,fire,fire%20truck,first%20responders,flames,flattened,flood,flooding,floods,forest%20fire,forest%20fires,hail,hailstorm,harm,hazard,hazardous,heat%20wave,hellfire,hijack,hijacker,hijacking,hostage,hostages,hurricane,injured,injuries,injury,inundated,inundation,landslide,lava,lightning,loud%20bang,mass%20murder,mass%20murderer,massacre,mayhem,meltdown,military,mudslide,natural%20disaster,nuclear%20disaster,nuclear%20reactor,obliterate,obliterated,obliteration,oil%20spill,outbreak,pandemonium,panic,panicking,police,quarantine,quarantined,radiation%20emergency,rainstorm,razed,refugees,rescue,rescued,rescuers,riot,rioting,rubble,ruin,sandstorm,screamed,screaming,screams,seismic,sinkhole,sinking,siren,sirens,smoke,snowstorm,storm,stretcher,structural%20failure,suicide%20bomb,suicide%20bomber,suicide%20bombing,sunk,survive,survived,survivors,terrorism,terrorist,threat,thunder,thunderstorm,tornado,tragedy,trapped,trauma,traumatised,trouble,tsunami,twister,typhoon,upheaval,violent%20storm,volcano,war%20zone,weapon,weapons,whirlwind,wild%20fires,wildfire,windstorm,wounded,wounds,wreck,wreckage,wrecked,NaN,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,1

In [9]:
testIDs = test.id
newTest = pipeline(test,False)
print(newTest.shape)
newTest.head(1)

(3263, 3443)


,len,words,numbers,sent,hashtags,mentions,links,density,punctuation,upper,ablaze,accident,aftershock,airplane%20accident,ambulance,annihilated,annihilation,apocalypse,armageddon,army,arson,arsonist,attack,attacked,avalanche,battle,bioterror,bioterrorism,blaze,blazing,bleeding,blew%20up,blight,blizzard,blood,bloody,blown%20up,body%20bag,body%20bagging,body%20bags,bomb,bombed,bombing,bridge%20collapse,buildings%20burning,buildings%20on%20fire,burned,burning,burning%20buildings,bush%20fires,casualties,casualty,catastrophe,catastrophic,chemical%20emergency,cliff%20fall,collapse,collapsed,collide,collided,collision,crash,crashed,crush,crushed,curfew,cyclone,damage,danger,dead,death,deaths,debris,deluge,deluged,demolish,demolished,demolition,derail,derailed,derailment,desolate,desolation,destroy,destroyed,destruction,detonate,detonation,devastated,devastation,disaster,displaced,drought,drown,drowned,drowning,dust%20storm,earthquake,electrocute,electrocuted,emergency,emergency%20plan,emergency%20services,engulfed,epicentre,evacuate,evacuated,evacuation,explode,exploded,explosion,eyewitness,famine,fatal,fatalities,fatality,fear,fire,fire%20truck,first%20responders,flames,flattened,flood,flooding,floods,forest%20fire,forest%20fires,hail,hailstorm,harm,hazard,hazardous,heat%20wave,hellfire,hijack,hijacker,hijacking,hostage,hostages,hurricane,injured,injuries,injury,inundated,inundation,landslide,lava,lightning,loud%20bang,mass%20murder,mass%20murderer,massacre,mayhem,meltdown,military,mudslide,natural%20disaster,nuclear%20disaster,nuclear%20reactor,obliterate,obliterated,obliteration,oil%20spill,outbreak,pandemonium,panic,panicking,police,quarantine,quarantined,radiation%20emergency,rainstorm,razed,refugees,rescue,rescued,rescuers,riot,rioting,rubble,ruin,sandstorm,screamed,screaming,screams,seismic,sinkhole,sinking,siren,sirens,smoke,snowstorm,storm,stretcher,structural%20failure,suicide%20bomb,suicide%20bomber,suicide%20bombing,sunk,survive,survived,survivors,terrorism,terrorist,threat,thunder,thunderstorm,tornado,tragedy,trapped,trauma,traumatised,trouble,tsunami,twister,typhoon,upheaval,violent%20storm,volcano,war%20zone,weapon,weapons,whirlwind,wild%20fires,wildfire,windstorm,wounded,wounds,wreck,wreckage,wrecked,NaN,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,

In [10]:
X_tr,X_te,y_tr,y_te = train_test_split(newTrain.drop('target',axis=1),
                                       newTrain['target'])
X_tr.shape

(4905, 3443)

In [11]:
mnBayes = MultinomialNB(alpha = 0.15).fit(X_tr,y_tr)
print(mnBayes.score(X_tr,y_tr))
print(mnBayes.score(X_te,y_te))

0.854026503567788
0.7993883792048929


In [12]:
svm = SVC(C=5,gamma=0.04).fit(scipy.sparse.csr_matrix(X_tr.values),y_tr)
print(svm.score(X_tr,y_tr))
print(svm.score(X_te,y_te))

0.8601427115188583
0.7926605504587156
